### Tugas Besar B - IF3270 Pembelajaran Mesin
Authors:

13520016 Gagas Praharsa Bahar

13520065 Rayhan Kinan Muhannad

13520081 Andhika Arta Aryanto

13520101 Aira Thalca Avila Putra

#### Install Library




In [39]:
%pip install numpy
%pip install typing
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### Import Library

In [40]:
from __future__ import annotations
import numpy as np
import time
from typing import NamedTuple, Callable
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


### Neural Network

#### Class Row

In [41]:
class Row(NamedTuple):
    weight: np.ndarray

#### Class Activation Function

In [42]:
class ActivationFunction(NamedTuple):
    function: Callable[[np.ndarray], np.ndarray]
    derivative_output: Callable[[np.ndarray, np.ndarray], np.ndarray]

    def __call__(self, o: np.ndarray) -> np.ndarray:
        return self.function(o)

    def get_derivative_output(self, o: np.ndarray, t: np.ndarray) -> np.ndarray:
        return self.derivative_output(o, t)


class LinearActivationFunction(ActivationFunction):
    def __new__(cls) -> LinearActivationFunction:
        # Inner Function
        def linear(x: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda x: x)(x))

        # Inner Function
        def derivative_output_linear(o: np.ndarray, _: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda _: 1)(o))

        self = super(LinearActivationFunction, cls).__new__(
            cls,
            linear,
            derivative_output_linear,
        )

        return self


class ReLUActivationFunction(ActivationFunction):
    def __new__(cls) -> ReLUActivationFunction:
        # Inner Function
        def relu(x: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda x: max(0, x))(x))

        # Inner Function
        def derivative_output_relu(o: np.ndarray, _: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda o: 1 if o > 0 else 0)(o))

        self = super(ReLUActivationFunction, cls).__new__(
            cls,
            relu,
            derivative_output_relu,
        )

        return self


class SigmoidActivationFunction(ActivationFunction):
    def __new__(cls) -> SigmoidActivationFunction:
        # Inner Function
        def sigmoid(x: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda x: 1 / (1 + np.exp(-x)))(x))

        # Inner Function
        def derivative_output_sigmoid(o: np.ndarray, _: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda o: o * (1 - o))(o))

        self = super(SigmoidActivationFunction, cls).__new__(
            cls,
            sigmoid,
            derivative_output_sigmoid,
        )

        return self


class SoftmaxActivationFunction(ActivationFunction):
    def __new__(cls) -> SoftmaxActivationFunction:
        # Inner Function
        def softmax(x: np.ndarray) -> np.ndarray:
            shift_x = np.array(x - np.max(x))  # Normalized
            exps = np.exp(shift_x)
            sums = np.sum(exps, axis=1, keepdims=True)
            return np.array(exps / sums)

        def derivative_output_softmax(o: np.ndarray, t: np.ndarray) -> np.ndarray:
            return np.array(np.subtract(o, t))

        self = super(SoftmaxActivationFunction, cls).__new__(
            cls,
            softmax,
            derivative_output_softmax,
        )

        return self

#### Class Layer

In [43]:
class Layer(NamedTuple):
    list_of_row: list[Row]
    activation_function: ActivationFunction

    def get_output(self, x: np.ndarray) -> np.ndarray:
        array_of_weight = np.array(
            [row.weight for row in self.list_of_row]
        )
        weighted_x = np.array(np.dot(array_of_weight.T, x))
        activated_x = self.activation_function(weighted_x)

        return activated_x

    def get_batch_output(self, batch_x: np.ndarray) -> np.ndarray:
        array_of_weight = np.array(
            [row.weight for row in self.list_of_row]
        )
        weighted_batch_x = np.array(np.dot(batch_x, array_of_weight))
        activated_batch_x = self.activation_function(weighted_batch_x)

        return activated_batch_x

    def get_weight(self) -> np.ndarray:
        return np.array(
            [row.weight for row in self.list_of_row]
        )

    def get_updated_weight(self, delta_weight: np.ndarray) -> Layer:
        array_of_weight = np.array(
            [row.weight for row in self.list_of_row]
        )

        new_array_of_weight = np.add(array_of_weight, delta_weight)
        list_of_row: list[Row] = [
            Row(weight) for weight in new_array_of_weight
        ]

        return Layer(list_of_row, self.activation_function)

#### Class Neural Network

In [44]:
class NeuralNetwork(NamedTuple):
    list_of_layer: list[Layer]

    def get_output(self, x: np.ndarray) -> np.ndarray:
        x_copy: np.ndarray = x.copy()

        for layer in self.list_of_layer:
            bias = np.ones(1)
            biased_x_copy = np.insert(x_copy, 0, bias, axis=0)
            x_copy = layer.get_output(biased_x_copy)

        return x_copy

    def get_batch_output(self, batch_x: np.ndarray) -> np.ndarray:
        N = batch_x.shape[0]
        batch_x_copy: np.ndarray = batch_x.copy()

        for layer in self.list_of_layer:
            bias = np.ones(N)
            biased_batch_x_copy = np.array(np.c_[bias, batch_x_copy])
            batch_x_copy = layer.get_batch_output(biased_batch_x_copy)

        return batch_x_copy

    def get_all_output(self, x: np.ndarray) -> list[np.ndarray]:
        x_copy: np.ndarray = x.copy()
        all_output: list[np.ndarray] = []

        for layer in self.list_of_layer:
            bias = np.ones(1)
            biased_x_copy = np.insert(x_copy, 0, bias, axis=0)
            x_copy = layer.get_output(biased_x_copy)
            all_output.append(x_copy)

        return all_output

    def get_all_batch_output(self, batch_x: np.ndarray) -> list[np.ndarray]:
        N = batch_x.shape[0]
        batch_x_copy: np.ndarray = batch_x.copy()
        all_output: list[np.ndarray] = []

        for layer in self.list_of_layer:
            bias = np.ones(N)
            biased_batch_x_copy = np.array(np.c_[bias, batch_x_copy])
            batch_x_copy = layer.get_batch_output(biased_batch_x_copy)
            all_output.append(batch_x_copy)

        return all_output

    def get_weight(self) -> list[np.ndarray]:
        return [layer.get_weight() for layer in self.list_of_layer]


### Backpropagation

#### Class Error Function

In [45]:
class ErrorFunction(NamedTuple):
    function: Callable[[np.ndarray, np.ndarray], float]

    def __call__(self, o: np.ndarray, t: np.ndarray) -> float:
        return self.function(o, t)


class SumOfSquaredErrorFunction(ErrorFunction):
    def __new__(cls) -> SumOfSquaredErrorFunction:
        # Inner Function
        def sum_of_squared_error(o: np.ndarray, t: np.ndarray) -> float:
            return np.sum(np.sum(np.square(o - t), axis=1) / 2, axis=0)

        self = super(SumOfSquaredErrorFunction, cls).__new__(
            cls,
            sum_of_squared_error,
        )

        return self


class CrossEntropyErrorFunction(ErrorFunction):
    def __new__(cls) -> CrossEntropyErrorFunction:
        # Inner Function
        def cross_entropy_error(o: np.ndarray, t: np.ndarray) -> float:
            return np.sum(-np.sum(t * np.log(o), axis=1), axis=0)

        self = super(CrossEntropyErrorFunction, cls).__new__(
            cls,
            cross_entropy_error,
        )

        return self

#### Class Mini Batch

In [46]:
class MiniBatch(NamedTuple):
    neural_network: NeuralNetwork
    partitioned_learning_data: np.ndarray
    partitioned_learning_target: np.ndarray

    def learn(self, learning_rate: float) -> None:
        list_of_output = self.neural_network.get_all_batch_output(
            self.partitioned_learning_data
        )
        previous_delta_error: np.ndarray = None
        new_layer: list[Layer] = [
            None for _ in range(len(list_of_output))
        ]

        for i in range(len(list_of_output) - 1, -1, -1):
            raw_x = self.partitioned_learning_data if i == 0 else list_of_output[i - 1]
            x = np.array(np.c_[np.ones(raw_x.shape[0]), raw_x])

            o = list_of_output[i]
            t = self.partitioned_learning_target
            derivated_output = self.neural_network.list_of_layer[i].activation_function.get_derivative_output(
                o, t
            )

            # Output Layer
            if i == len(list_of_output) - 1:
                delta_error = -np.multiply(
                    np.subtract(o, t),
                    derivated_output
                ) if type(self.neural_network.list_of_layer[i].activation_function) is not SoftmaxActivationFunction else -derivated_output
                delta_weight = np.array(
                    np.dot(learning_rate, np.dot(x.T, delta_error))
                )
                new_layer[i] = self.neural_network.list_of_layer[i].get_updated_weight(
                    delta_weight
                )
                previous_delta_error = delta_error

            # Hidden Layer
            else:
                output_weight = self.neural_network.list_of_layer[i + 1].get_weight()[
                    1:
                ]
                delta_error = np.multiply(
                    np.dot(
                        previous_delta_error, output_weight.T
                    ),
                    derivated_output
                )
                delta_weight = np.array(
                    np.dot(learning_rate, np.dot(x.T, delta_error))
                )
                new_layer[i] = self.neural_network.list_of_layer[i].get_updated_weight(
                    delta_weight
                )
                previous_delta_error = delta_error

        for i in range(len(new_layer)):
            self.neural_network.list_of_layer[i] = new_layer[i]

#### Class Backpropagation

In [47]:
class Backpropagation(NamedTuple):
    neural_network: NeuralNetwork
    learning_data: np.ndarray
    learning_target: np.ndarray

    def learn(self, learning_rate: float, mini_batch_size: int, max_iter: int, threshold: float, error_function: ErrorFunction) -> None:
        data_length = self.learning_data.shape[0]
        current_error = np.inf
        index = 0

        # Until the error is less than or equal to the threshold or the maximum iteration is reached
        while current_error > threshold and index < max_iter:
            start_time = time.time()

            # Get Output
            current_output = self.neural_network.get_batch_output(
                self.learning_data
            )

            # Mini-Batch Learning
            start_index = 0
            while start_index < data_length:
                end_index = min(start_index + mini_batch_size, data_length)
                partitioned_learning_data = self.learning_data[start_index:end_index]
                partitioned_learning_target = self.learning_target[start_index:end_index]

                mini_batch = MiniBatch(
                    self.neural_network,
                    partitioned_learning_data,
                    partitioned_learning_target,
                )
                mini_batch.learn(learning_rate)
                start_index += mini_batch_size

            # Get Error
            current_error = error_function(
                current_output,
                self.learning_target
            )

            finish_time = time.time()

            print(
                f"Epoch {index + 1}\t|\tError: {round(current_error, 4)}\t|\tTime: {round(1000 * (finish_time - start_time), 4)} ms"
            )

            index += 1


### File System

In [48]:
class FileSystem:
    @staticmethod
    def load_from_file(path: str) -> NeuralNetwork:
        with open(path, "r") as file:
            input_size = int(file.readline().strip())
            num_of_layers = int(file.readline().strip())

            prev_num_of_perceptrons = input_size
            list_of_layer: list[Layer] = []
            latest_activation_function_type: str = None

            for _ in range(num_of_layers):
                num_of_perceptrons = int(file.readline().strip())
                list_of_weight_row: list[Row] = []

                for _ in range(prev_num_of_perceptrons + 1):
                    weight = np.array(
                        list(map(float, file.readline().strip().split()))
                    )
                    row = Row(weight)
                    list_of_weight_row.append(row)

                activation_function_type = file.readline().strip()

                if latest_activation_function_type == "softmax":
                    raise NotImplementedError()

                activation_function: ActivationFunction
                if activation_function_type == "linear":
                    activation_function = LinearActivationFunction()
                elif activation_function_type == "relu":
                    activation_function = ReLUActivationFunction()
                elif activation_function_type == "sigmoid":
                    activation_function = SigmoidActivationFunction()
                elif activation_function_type == "softmax":
                    activation_function = SoftmaxActivationFunction()
                else:
                    raise NotImplementedError()

                layer = Layer(list_of_weight_row, activation_function)
                list_of_layer.append(layer)

                prev_num_of_perceptrons = num_of_perceptrons
                latest_activation_function_type = activation_function_type

            neural_network = NeuralNetwork(list_of_layer)

            return neural_network

    @staticmethod
    def save_to_file(neural_network: NeuralNetwork, path: str) -> None:
        with open(path, "w") as file:
            input_weight = neural_network.list_of_layer[0].get_weight()
            input_size = input_weight.shape[0] - 1
            file.write(f"{input_size}\n")

            num_of_layers = len(neural_network.list_of_layer)
            file.write(f"{num_of_layers}\n")

            for layer in neural_network.list_of_layer:
                weight = layer.get_weight()
                num_of_perceptrons = weight.shape[1]
                file.write(f"{num_of_perceptrons}\n")

                for i in range(weight.shape[0]):
                    for j in range(weight.shape[1]):
                        file.write(f"{weight[i][j]} ")
                    file.write("\n")

                activation_function_type: str = None
                if type(layer.activation_function) is LinearActivationFunction:
                    activation_function_type = "linear"
                elif type(layer.activation_function) is ReLUActivationFunction:
                    activation_function_type = "relu"
                elif type(layer.activation_function) is SigmoidActivationFunction:
                    activation_function_type = "sigmoid"
                elif type(layer.activation_function) is SoftmaxActivationFunction:
                    activation_function_type = "softmax"
                else:
                    raise NotImplementedError()

                file.write(f"{activation_function_type}\n")

    @staticmethod
    def learn_from_file(path: str) -> NeuralNetwork:
        with open(path, "r") as file:
            input_size = int(file.readline().strip())
            num_of_layers = int(file.readline().strip())

            prev_num_of_perceptrons = input_size
            list_of_layer: list[Layer] = []
            latest_activation_function_type: str = None

            for _ in range(num_of_layers):
                num_of_perceptrons = int(file.readline().strip())
                list_of_weight_row: list[Row] = []

                for _ in range(prev_num_of_perceptrons + 1):
                    weight = np.array(
                        list(map(float, file.readline().strip().split()))
                    )
                    row = Row(weight)
                    list_of_weight_row.append(row)

                activation_function_type = file.readline().strip()

                if latest_activation_function_type == "softmax":
                    raise NotImplementedError()

                activation_function: ActivationFunction
                if activation_function_type == "linear":
                    activation_function = LinearActivationFunction()
                elif activation_function_type == "relu":
                    activation_function = ReLUActivationFunction()
                elif activation_function_type == "sigmoid":
                    activation_function = SigmoidActivationFunction()
                elif activation_function_type == "softmax":
                    activation_function = SoftmaxActivationFunction()
                else:
                    raise NotImplementedError()

                layer = Layer(list_of_weight_row, activation_function)
                list_of_layer.append(layer)

                prev_num_of_perceptrons = num_of_perceptrons
                latest_activation_function_type = activation_function_type

            initial_neural_network = NeuralNetwork(list_of_layer)

            test_case_size = int(file.readline().strip())
            input_array: list[np.ndarray] = []
            target_array: list[np.ndarray] = []

            for _ in range(test_case_size):
                input_vector = list(
                    map(float, file.readline().strip().split())
                )

                input_array.append(np.array(input_vector))

            for _ in range(test_case_size):
                target_vector = list(
                    map(float, file.readline().strip().split())
                )
                target_array.append(np.array(target_vector))

            learning_rate = float(file.readline().strip())
            mini_batch_size = int(file.readline().strip())
            max_iter = int(file.readline().strip())
            threshold = float(file.readline().strip())

            backpropagation = Backpropagation(
                initial_neural_network,
                np.array(input_array),
                np.array(target_array)
            )

            error_function: ErrorFunction
            if latest_activation_function_type == "linear" or latest_activation_function_type == "relu" or latest_activation_function_type == "sigmoid":
                error_function = SumOfSquaredErrorFunction()
            elif latest_activation_function_type == "softmax":
                error_function = CrossEntropyErrorFunction()
            else:
                raise NotImplementedError()

            backpropagation.learn(
                learning_rate, mini_batch_size, max_iter, threshold, error_function
            )

            return backpropagation.neural_network


### Test Case

#### Linear Small LR

In [49]:
neural_network = FileSystem.learn_from_file(
    "../test-cases/txt/linear_small_lr.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()
        

Epoch 1	|	Error: 0.665	|	Time: 0.3772 ms
[[ 0.1012  0.3006  0.1991]
 [ 0.4024  0.201  -0.7019]
 [ 0.1018 -0.799   0.4987]]



#### Linear Two Iteration

In [50]:
neural_network = FileSystem.learn_from_file(
    "../test-cases/txt/linear_two_iteration.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.665	|	Time: 0.2098 ms
Epoch 2	|	Error: 0.1818	|	Time: 0.1888 ms
[[ 0.166  0.338  0.153]
 [ 0.502  0.226 -0.789]
 [ 0.214 -0.718  0.427]]



#### Linear

In [51]:
neural_network = FileSystem.learn_from_file(
    "../test-cases/txt/linear.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.665	|	Time: 0.2828 ms
[[ 0.22  0.36  0.11]
 [ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]



#### MLP

In [52]:
neural_network = FileSystem.learn_from_file(
    "../test-cases/txt/mlp.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.8186	|	Time: 0.3021 ms
[[ 0.07115   0.1403  ]
 [ 0.42885  -0.4403  ]
 [ 0.685575  0.77015 ]]

[[ 0.021     0.1945  ]
 [ 0.39605  -0.500275]
 [ 0.6131    0.79395 ]]



#### ReLU

In [53]:
neural_network = FileSystem.learn_from_file(
    "../test-cases/txt/relu.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.1463	|	Time: 0.206 ms
[[ 0.105   0.19    0.25  ]
 [ 0.395  -0.49    0.575 ]
 [ 0.7025  0.795  -0.85  ]]



#### Sigmoid Mini Batch

In [54]:
neural_network = FileSystem.learn_from_file(
    "../test-cases/txt/sigmoid_mini_batch_GD.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 1.0857	|	Time: 0.2749 ms
Epoch 2	|	Error: 1.0796	|	Time: 0.1318 ms
Epoch 3	|	Error: 1.0736	|	Time: 0.1512 ms
Epoch 4	|	Error: 1.0676	|	Time: 0.1438 ms
Epoch 5	|	Error: 1.0617	|	Time: 0.1388 ms
Epoch 6	|	Error: 1.0558	|	Time: 0.2429 ms
Epoch 7	|	Error: 1.05	|	Time: 0.3011 ms
Epoch 8	|	Error: 1.0442	|	Time: 0.3378 ms
Epoch 9	|	Error: 1.0385	|	Time: 0.1192 ms
Epoch 10	|	Error: 1.0329	|	Time: 0.1209 ms
Epoch 11	|	Error: 1.0274	|	Time: 0.1323 ms
Epoch 12	|	Error: 1.022	|	Time: 0.211 ms
Epoch 13	|	Error: 1.0166	|	Time: 0.139 ms
Epoch 14	|	Error: 1.0114	|	Time: 0.1149 ms
Epoch 15	|	Error: 1.0062	|	Time: 0.1078 ms
Epoch 16	|	Error: 1.0011	|	Time: 0.107 ms
Epoch 17	|	Error: 0.9962	|	Time: 0.1032 ms
Epoch 18	|	Error: 0.9913	|	Time: 0.1061 ms
Epoch 19	|	Error: 0.9865	|	Time: 0.104 ms
Epoch 20	|	Error: 0.9818	|	Time: 0.1042 ms
Epoch 21	|	Error: 0.9772	|	Time: 0.1042 ms
Epoch 22	|	Error: 0.9726	|	Time: 0.1049 ms
Epoch 23	|	Error: 0.9682	|	Time: 0.1009 ms
Epoch 24	|	Error: 0.9639	|	

#### Sigmoid Stochastic

In [55]:
neural_network = FileSystem.learn_from_file(
    "../test-cases/txt/sigmoid_stochastic_GD.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 1.0857	|	Time: 0.299 ms
Epoch 2	|	Error: 1.0797	|	Time: 0.2759 ms
Epoch 3	|	Error: 1.0736	|	Time: 0.3028 ms
Epoch 4	|	Error: 1.0676	|	Time: 0.2742 ms
Epoch 5	|	Error: 1.0617	|	Time: 0.1941 ms
Epoch 6	|	Error: 1.0558	|	Time: 0.185 ms
Epoch 7	|	Error: 1.05	|	Time: 0.181 ms
Epoch 8	|	Error: 1.0442	|	Time: 0.232 ms
Epoch 9	|	Error: 1.0385	|	Time: 0.2091 ms
Epoch 10	|	Error: 1.0329	|	Time: 0.284 ms
Epoch 11	|	Error: 1.0274	|	Time: 0.3309 ms
Epoch 12	|	Error: 1.022	|	Time: 0.303 ms
Epoch 13	|	Error: 1.0166	|	Time: 0.1769 ms
Epoch 14	|	Error: 1.0113	|	Time: 0.1781 ms
Epoch 15	|	Error: 1.0061	|	Time: 0.1738 ms
Epoch 16	|	Error: 1.001	|	Time: 0.1729 ms
Epoch 17	|	Error: 0.996	|	Time: 0.174 ms
Epoch 18	|	Error: 0.9911	|	Time: 0.17 ms
Epoch 19	|	Error: 0.9863	|	Time: 0.169 ms
Epoch 20	|	Error: 0.9816	|	Time: 0.1869 ms
Epoch 21	|	Error: 0.9769	|	Time: 0.1721 ms
Epoch 22	|	Error: 0.9724	|	Time: 0.2861 ms
Epoch 23	|	Error: 0.9679	|	Time: 0.19 ms
Epoch 24	|	Error: 0.9636	|	Time: 0.17

#### Sigmoid

In [67]:
neural_network = FileSystem.learn_from_file(
    "../test-cases/txt/sigmoid.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.4466	|	Time: 0.6962 ms
[[0.10061658 0.19748577]
 [0.4        0.5       ]
 [0.90112145 0.09863434]]



#### Softmax Error Only

In [57]:
neural_network = FileSystem.learn_from_file(
    "../test-cases/txt/softmax_error_only.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.2204	|	Time: 0.3979 ms
[[ 0.38021839  0.71978161]
 [ 0.21978161 -0.91978161]]



#### Softmax

In [58]:
neural_network = FileSystem.learn_from_file(
    "../test-cases/txt/softmax.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 2.2933	|	Time: 0.1662 ms
[[ 0.11301357  0.18698643]
 [ 0.29539055 -0.39539055]
 [ 0.79810267  0.70189733]]



#### SSE Only

In [59]:
neural_network = FileSystem.learn_from_file(
    "../test-cases/txt/sse_only.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.25	|	Time: 0.2129 ms
[[0.495 0.505]
 [0.    0.   ]]



#### Learning Dataset Iris (using train-test split)

In [60]:
iris = load_iris()
learning_data = np.array(iris.data)
learning_target = np.array(iris.target)

one_hot_learning_target = np.zeros(
    (learning_target.size, learning_target.max() + 1)
)
one_hot_learning_target[np.arange(
    learning_target.size), learning_target] = 1

X_train, X_test, y_train, y_test = train_test_split(
    learning_data,
    one_hot_learning_target,
    test_size=0.2,
    random_state=42
)
numpy_X_train = np.array(X_train)
numpy_X_test = np.array(X_test)
numpy_y_train = np.array(y_train)
numpy_y_test = np.array(y_test)

neural_network = FileSystem.load_from_file(
    "../model/iris-raw.txt"
)
backpropagation = Backpropagation(
    neural_network,
    numpy_X_train,
    numpy_y_train
)

cross_entropy = CrossEntropyErrorFunction()

backpropagation.learn(
    learning_rate=0.01,
    mini_batch_size=1,
    max_iter=10000,
    threshold=0.05 * len(numpy_X_train),
    error_function=cross_entropy
)
print()

print("Weight:")
for weight in neural_network.get_weight():
    print(weight)
print()

y_pred = neural_network.get_batch_output(numpy_X_test)

print(f"Prediction:\n{y_pred}")
print()

print(f"Actual:\n{numpy_y_test}")
print()

error = cross_entropy(y_pred, numpy_y_test)
print(f"Error: {error} / {round(100 * error / len(numpy_X_test), 2)}%")

FileSystem.save_to_file(
    neural_network,
    "../model/iris-learn.txt"
)

Epoch 1	|	Error: 131.8335	|	Time: 17.2343 ms
Epoch 2	|	Error: 131.9638	|	Time: 17.3347 ms
Epoch 3	|	Error: 132.0501	|	Time: 15.7189 ms
Epoch 4	|	Error: 132.0797	|	Time: 14.9801 ms
Epoch 5	|	Error: 132.0887	|	Time: 15.6341 ms
Epoch 6	|	Error: 132.091	|	Time: 17.1158 ms
Epoch 7	|	Error: 132.0913	|	Time: 17.1549 ms
Epoch 8	|	Error: 132.0911	|	Time: 17.1201 ms
Epoch 9	|	Error: 132.0906	|	Time: 18.862 ms
Epoch 10	|	Error: 132.0901	|	Time: 18.6939 ms
Epoch 11	|	Error: 132.0896	|	Time: 16.9177 ms
Epoch 12	|	Error: 132.089	|	Time: 16.0849 ms
Epoch 13	|	Error: 132.0885	|	Time: 15.7051 ms
Epoch 14	|	Error: 132.0879	|	Time: 16.4571 ms
Epoch 15	|	Error: 132.0874	|	Time: 15.8339 ms
Epoch 16	|	Error: 132.0869	|	Time: 15.2419 ms
Epoch 17	|	Error: 132.0863	|	Time: 15.135 ms
Epoch 18	|	Error: 132.0858	|	Time: 15.1079 ms
Epoch 19	|	Error: 132.0853	|	Time: 15.5289 ms
Epoch 20	|	Error: 132.0848	|	Time: 15.9471 ms
Epoch 21	|	Error: 132.0842	|	Time: 15.955 ms
Epoch 22	|	Error: 132.0837	|	Time: 17.3562 ms
Ep

#### Comparison with scikit-learn

In [61]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris

In [62]:
iris = load_iris()
learning_data = np.array(iris.data)
learning_target = np.array(iris.target)
clf = MLPClassifier(hidden_layer_sizes=(8,8,3), activation='logistic', max_iter=10000, learning_rate='constant', learning_rate_init=0.01, random_state=1)
clf.fit(iris.data, iris.target)

MLPClassifier(activation='logistic', hidden_layer_sizes=(8, 8, 3),
              learning_rate_init=0.01, max_iter=10000, random_state=1)

In [63]:
y_pred = clf.predict(learning_data)
error_function = SumOfSquaredErrorFunction()
print("SKLearn MLPClassifier with 8,8,3 hidden layers and logistic activation function")
print(f"Prediction:\n{y_pred}")
print()
print(f"Actual:\n{learning_target}")
error = error_function(y_pred.reshape(150,1), learning_target.reshape(150,1))
print(f"Error: {error} / {round(100 * error / len(learning_data), 2)}%")

SKLearn MLPClassifier with 8,8,3 hidden layers and logistic activation function
Prediction:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]

Actual:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Error: 0.5 / 0.33%


In [64]:
iris = load_iris()

learning_data = np.array(iris.data)
learning_target = np.array(iris.target)
one_hot_learning_target = np.zeros(
    (learning_target.size, learning_target.max() + 1)
)
one_hot_learning_target[np.arange(
    learning_target.size), learning_target] = 1

learning_data = np.array(learning_data)
learning_target = np.array(one_hot_learning_target)
neural_network = FileSystem.load_from_file(
    "../model/iris-raw-3sigmoid.txt"
)
backpropagation = Backpropagation(
    neural_network,
    learning_data,
    learning_target
)
error_function = SumOfSquaredErrorFunction()
backpropagation.learn(
    learning_rate=0.01,
    mini_batch_size=1,
    max_iter=10000,
    threshold=0.05 * len(learning_data),
    error_function=error_function
)
print()
print("Weight:")
for weight in neural_network.get_weight():
    print(weight)
print()

Epoch 1	|	Error: 56.25	|	Time: 20.5309 ms
Epoch 2	|	Error: 53.5736	|	Time: 20.2079 ms
Epoch 3	|	Error: 52.0786	|	Time: 19.9292 ms
Epoch 4	|	Error: 51.2403	|	Time: 19.9289 ms
Epoch 5	|	Error: 50.764	|	Time: 19.604 ms
Epoch 6	|	Error: 50.4893	|	Time: 19.7787 ms
Epoch 7	|	Error: 50.3289	|	Time: 19.7241 ms
Epoch 8	|	Error: 50.234	|	Time: 20.313 ms
Epoch 9	|	Error: 50.1776	|	Time: 19.8839 ms
Epoch 10	|	Error: 50.1438	|	Time: 20.8809 ms
Epoch 11	|	Error: 50.1235	|	Time: 21.4748 ms
Epoch 12	|	Error: 50.1113	|	Time: 20.0613 ms
Epoch 13	|	Error: 50.104	|	Time: 20.1528 ms
Epoch 14	|	Error: 50.0997	|	Time: 19.7101 ms
Epoch 15	|	Error: 50.0972	|	Time: 19.9151 ms
Epoch 16	|	Error: 50.0958	|	Time: 19.7556 ms
Epoch 17	|	Error: 50.095	|	Time: 19.6521 ms
Epoch 18	|	Error: 50.0946	|	Time: 19.5191 ms
Epoch 19	|	Error: 50.0944	|	Time: 20.267 ms
Epoch 20	|	Error: 50.0943	|	Time: 19.742 ms
Epoch 21	|	Error: 50.0943	|	Time: 21.796 ms
Epoch 22	|	Error: 50.0943	|	Time: 22.3629 ms
Epoch 23	|	Error: 50.0943	|	Ti

In [65]:
y_pred = neural_network.get_batch_output(learning_data)

print(f"Prediction:\n{y_pred.round()}")
print()

print(f"Actual:\n{learning_target}")
print()

error = error_function(y_pred.round(), learning_target)
print(f"Error: {error} / {round(100 * error / len(learning_data), 2)}%")

Prediction:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0.

In [66]:
FileSystem.save_to_file(neural_network, "../model/iris-learn-full.txt")